# Meccanismi di attenzione e trasformatori

Uno dei principali svantaggi delle reti ricorrenti è che tutte le parole in una sequenza hanno lo stesso impatto sul risultato. Questo causa prestazioni subottimali con i modelli standard encoder-decoder LSTM per compiti di sequenza a sequenza, come il Riconoscimento di Entità Nominate e la Traduzione Automatica. In realtà, specifiche parole nella sequenza di input spesso hanno un impatto maggiore sugli output sequenziali rispetto ad altre.

Consideriamo un modello di sequenza a sequenza, come la traduzione automatica. Questo viene implementato con due reti ricorrenti, dove una rete (**encoder**) comprime la sequenza di input in uno stato nascosto, e un'altra rete, il **decoder**, espande questo stato nascosto nel risultato tradotto. Il problema con questo approccio è che lo stato finale della rete ha difficoltà a ricordare l'inizio di una frase, causando una scarsa qualità del modello con frasi lunghe.

I **meccanismi di attenzione** forniscono un mezzo per pesare l'impatto contestuale di ciascun vettore di input su ciascuna previsione di output dell'RNN. Questo viene implementato creando scorciatoie tra gli stati intermedi dell'RNN di input e l'RNN di output. In questo modo, quando si genera il simbolo di output $y_t$, si tiene conto di tutti gli stati nascosti di input $h_i$, con diversi coefficienti di peso $\alpha_{t,i}$. 

![Immagine che mostra un modello encoder/decoder con uno strato di attenzione additiva](../../../../../translated_images/it/encoder-decoder-attention.7a726296894fb567.webp)
*Il modello encoder-decoder con meccanismo di attenzione additiva in [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), citato da [questo post sul blog](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

La matrice di attenzione $\{\alpha_{i,j}\}$ rappresenta il grado in cui certe parole di input influenzano la generazione di una determinata parola nella sequenza di output. Di seguito è riportato un esempio di tale matrice:

![Immagine che mostra un allineamento campione trovato da RNNsearch-50, tratto da Bahdanau - arviz.org](../../../../../translated_images/it/bahdanau-fig3.09ba2d37f202a6af.webp)

*Figura tratta da [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

I meccanismi di attenzione sono responsabili di gran parte dello stato dell'arte attuale o quasi attuale nell'elaborazione del linguaggio naturale. Tuttavia, aggiungere l'attenzione aumenta notevolmente il numero di parametri del modello, il che ha portato a problemi di scalabilità con gli RNN. Una limitazione chiave nella scalabilità degli RNN è che la natura ricorrente dei modelli rende difficile batchare e parallelizzare l'addestramento. In un RNN, ogni elemento di una sequenza deve essere elaborato in ordine sequenziale, il che significa che non può essere facilmente parallelizzato.

L'adozione dei meccanismi di attenzione, combinata con questa limitazione, ha portato alla creazione dei modelli Transformer, ora lo stato dell'arte, che conosciamo e utilizziamo oggi, da BERT a OpenGPT3.

## Modelli Transformer

Invece di trasmettere il contesto di ciascuna previsione precedente al passaggio di valutazione successivo, i **modelli Transformer** utilizzano **codifiche posizionali** e **attenzione** per catturare il contesto di un dato input all'interno di una finestra di testo fornita. L'immagine seguente mostra come le codifiche posizionali con attenzione possano catturare il contesto all'interno di una determinata finestra.

![GIF animata che mostra come vengono effettuate le valutazioni nei modelli Transformer.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

Poiché ogni posizione di input viene mappata indipendentemente a ciascuna posizione di output, i Transformer possono parallelizzare meglio rispetto agli RNN, il che consente modelli linguistici molto più grandi ed espressivi. Ogni testa di attenzione può essere utilizzata per apprendere diverse relazioni tra le parole, migliorando i compiti di elaborazione del linguaggio naturale a valle.

## Costruire un semplice modello Transformer

Keras non contiene un livello Transformer integrato, ma possiamo costruirne uno nostro. Come in precedenza, ci concentreremo sulla classificazione del testo del dataset AG News, ma vale la pena menzionare che i modelli Transformer mostrano i migliori risultati nei compiti NLP più difficili.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Nuovi livelli in Keras devono sottoclassare la classe `Layer` e implementare il metodo `call`. Iniziamo con il livello **Positional Embedding**. Useremo [alcuni codici dalla documentazione ufficiale di Keras](https://keras.io/examples/nlp/text_classification_with_transformer/). Assumeremo che riempiamo tutte le sequenze di input fino alla lunghezza `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Questo livello è composto da due livelli `Embedding`: uno per l'incorporazione dei token (nel modo che abbiamo discusso in precedenza) e uno per le posizioni dei token. Le posizioni dei token vengono create come una sequenza di numeri naturali da 0 a `maxlen` utilizzando `tf.range`, e poi passate attraverso il livello di embedding. I due vettori di embedding risultanti vengono quindi sommati, producendo una rappresentazione posizionale incorporata dell'input con forma `maxlen`$\times$`embed_dim`.

Ora, implementiamo il blocco transformer. Prenderà in ingresso l'output del livello di embedding definito in precedenza:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Ora siamo pronti a definire il modello completo del transformer:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## Modelli Transformer BERT

**BERT** (Bidirectional Encoder Representations from Transformers) è una rete transformer multilivello molto grande con 12 livelli per *BERT-base* e 24 per *BERT-large*. Il modello viene inizialmente pre-addestrato su un ampio corpus di dati testuali (WikiPedia + libri) utilizzando un addestramento non supervisionato (predizione di parole mascherate in una frase). Durante la fase di pre-addestramento, il modello acquisisce un livello significativo di comprensione del linguaggio che può essere poi sfruttato con altri dataset attraverso il fine tuning. Questo processo è chiamato **apprendimento trasferibile**.

![immagine da http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/it/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Esistono molte varianti delle architetture Transformer, tra cui BERT, DistilBERT, BigBird, OpenGPT3 e altre, che possono essere ottimizzate.

Vediamo come possiamo utilizzare un modello BERT pre-addestrato per risolvere il nostro tradizionale problema di classificazione delle sequenze. Prenderemo spunto e parte del codice dalla [documentazione ufficiale](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Per caricare modelli pre-addestrati, utilizzeremo **Tensorflow hub**. Per prima cosa, carichiamo il vettorizzatore specifico di BERT:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

È importante utilizzare lo stesso vettorizzatore che è stato usato per addestrare la rete originale. Inoltre, il vettorizzatore BERT restituisce tre componenti:
* `input_word_ids`, che è una sequenza di numeri di token per la frase di input
* `input_mask`, che indica quale parte della sequenza contiene l'input effettivo e quale è il padding. È simile alla maschera prodotta dal livello `Masking`
* `input_type_ids` viene utilizzato per compiti di modellazione del linguaggio e permette di specificare due frasi di input in una sola sequenza.

Successivamente, possiamo istanziare l'estrattore di caratteristiche BERT:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Quindi, il livello BERT restituisce una serie di risultati utili:
* `pooled_output` è il risultato della media di tutti i token nella sequenza. Puoi considerarlo come un embedding semantico intelligente dell'intera rete. È equivalente all'output del livello `GlobalAveragePooling1D` nel nostro modello precedente.
* `sequence_output` è l'output dell'ultimo livello transformer (corrisponde all'output di `TransformerBlock` nel nostro modello sopra).
* `encoder_outputs` sono gli output di tutti i livelli transformer. Poiché abbiamo caricato un modello BERT a 4 livelli (come probabilmente puoi intuire dal nome, che contiene `4_H`), ha 4 tensori. L'ultimo è lo stesso di `sequence_output`.

Ora definiremo il modello di classificazione end-to-end. Utilizzeremo la *definizione funzionale del modello*, in cui definiamo l'input del modello e poi forniamo una serie di espressioni per calcolarne l'output. Inoltre, renderemo i pesi del modello BERT non addestrabili e addestreremo solo il classificatore finale:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Nonostante il fatto che ci siano pochi parametri allenabili, il processo è piuttosto lento, perché l'estrattore di caratteristiche di BERT è computazionalmente pesante. Sembra che non siamo riusciti a ottenere un'accuratezza ragionevole, sia per mancanza di allenamento, sia per mancanza di parametri del modello.

Proviamo a sbloccare i pesi di BERT e ad allenarlo anche. Questo richiede un tasso di apprendimento molto basso e una strategia di allenamento più attenta con **warmup**, utilizzando l'ottimizzatore **AdamW**. Utilizzeremo il pacchetto `tf-models-official` per creare l'ottimizzatore:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Come puoi vedere, l'addestramento procede piuttosto lentamente - ma potresti voler sperimentare e addestrare il modello per alcuni cicli (5-10) e vedere se riesci a ottenere il miglior risultato rispetto agli approcci che abbiamo utilizzato prima.

## Libreria Huggingface Transformers

Un altro modo molto comune (e un po' più semplice) per utilizzare i modelli Transformer è [HuggingFace package](https://github.com/huggingface/), che fornisce blocchi semplici per diversi compiti di NLP. È disponibile sia per Tensorflow che per PyTorch, un altro framework di reti neurali molto popolare.

> **Nota**: Se non sei interessato a vedere come funziona la libreria Transformers - puoi saltare alla fine di questo notebook, perché non vedrai nulla di sostanzialmente diverso da ciò che abbiamo fatto sopra. Ripeteremo gli stessi passaggi di addestramento del modello BERT utilizzando una libreria diversa e un modello sostanzialmente più grande. Pertanto, il processo comporta un addestramento piuttosto lungo, quindi potresti voler semplicemente dare un'occhiata al codice.

Vediamo come il nostro problema può essere risolto utilizzando [Huggingface Transformers](http://huggingface.co).


La prima cosa da fare è scegliere il modello che utilizzeremo. Oltre ad alcuni modelli integrati, Huggingface contiene un [repository online di modelli](https://huggingface.co/models), dove puoi trovare molti altri modelli pre-addestrati dalla comunità. Tutti questi modelli possono essere caricati e utilizzati semplicemente fornendo il nome del modello. Tutti i file binari necessari per il modello verranno scaricati automaticamente.

In alcuni casi potrebbe essere necessario caricare i propri modelli, nel qual caso è possibile specificare la directory che contiene tutti i file rilevanti, inclusi i parametri per il tokenizer, il file `config.json` con i parametri del modello, i pesi binari, ecc.

Dal nome del modello, possiamo istanziare sia il modello che il tokenizer. Iniziamo con un tokenizer:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

L'oggetto `tokenizer` contiene la funzione `encode` che può essere utilizzata direttamente per codificare il testo:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Possiamo anche utilizzare il tokenizer per codificare una sequenza in un modo adatto per essere passato al modello, cioè includendo i campi `token_ids`, `input_mask`, ecc. Possiamo anche specificare che vogliamo tensori Tensorflow fornendo l'argomento `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

Nel nostro caso, utilizzeremo un modello BERT pre-addestrato chiamato `bert-base-uncased`. *Uncased* indica che il modello non fa distinzione tra maiuscole e minuscole.

Durante l'addestramento del modello, dobbiamo fornire una sequenza tokenizzata come input, e quindi progetteremo una pipeline di elaborazione dei dati. Poiché `tokenizer.encode` è una funzione Python, utilizzeremo lo stesso approccio dell'unità precedente chiamandola tramite `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Ora possiamo caricare il modello effettivo utilizzando il pacchetto `BertForSequenceClassification`. Questo garantisce che il nostro modello abbia già un'architettura necessaria per la classificazione, incluso il classificatore finale. Vedrai un messaggio di avviso che indica che i pesi del classificatore finale non sono inizializzati e che il modello richiederebbe un pre-addestramento - va benissimo, perché è esattamente ciò che stiamo per fare!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Come puoi vedere da `summary()`, il modello contiene quasi 110 milioni di parametri! Presumibilmente, se vogliamo un semplice compito di classificazione su un dataset relativamente piccolo, non vogliamo addestrare il livello base di BERT:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Ora siamo pronti per iniziare l'addestramento!

> **Nota**: L'addestramento di un modello BERT completo può richiedere molto tempo! Per questo motivo, lo addestreremo solo per i primi 32 batch. Questo serve solo a mostrare come viene configurato l'addestramento del modello. Se sei interessato a provare l'addestramento completo, basta rimuovere i parametri `steps_per_epoch` e `validation_steps`, e prepararti ad aspettare!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Se aumenti il numero di iterazioni e aspetti abbastanza a lungo, e alleni per diversi epoch, puoi aspettarti che la classificazione con BERT ci dia la migliore accuratezza! Questo perché BERT già comprende molto bene la struttura della lingua, e dobbiamo solo perfezionare il classificatore finale. Tuttavia, poiché BERT è un modello grande, l'intero processo di addestramento richiede molto tempo e necessita di una potenza computazionale significativa! (GPU, e preferibilmente più di una).

> **Nota:** Nel nostro esempio, abbiamo utilizzato uno dei modelli BERT pre-addestrati più piccoli. Esistono modelli più grandi che probabilmente offrono risultati migliori.


## Punti chiave

In questa unità, abbiamo esaminato architetture di modelli molto recenti basate sui **transformer**. Le abbiamo applicate al nostro compito di classificazione del testo, ma allo stesso modo i modelli BERT possono essere utilizzati per l'estrazione di entità, la risposta a domande e altri compiti di NLP.

I modelli transformer rappresentano lo stato dell'arte attuale nel NLP e, nella maggior parte dei casi, dovrebbero essere la prima soluzione con cui iniziare a sperimentare quando si implementano soluzioni NLP personalizzate. Tuttavia, comprendere i principi fondamentali delle reti neurali ricorrenti discussi in questo modulo è estremamente importante se si desidera costruire modelli neurali avanzati.



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
